<a href="https://colab.research.google.com/github/pachterlab/JBMMCKP_2023/blob/main/notebooks/cdx_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

connect to R runtime above with `Runtime > Change Runtime Type`

In [ ]:
system("git clone https://github.com/pachterlab/JBMMCKP_2023.git")
setwd("JBMMCKP_2023")

Install `magick` and `fastcluster` on Colab

In [ ]:
# Installation of  `magick` is req'd for some BioC packages,
# but installation is halted on Colab
system('add-apt-repository -y ppa:cran/imagemagick')
system('apt-get update')
system("apt-get install libmagick++-dev")
system("sudo apt-get install libgmp-dev")

install.packages("magick")
install.packages("fastcluster")

Install Bioconductor packages

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.19")

# Install Bioconductor packages
BiocManager::install("SpatialExperiment")
BiocManager::install("bluster")
BiocManager::install("scater")
BiocManager::install("mbkmeans")

In [ ]:
 devtools::install_github("pachterlab/concordexR")

Load  `concordexR` and some extra libraries for plotting and utility functions.

In [ ]:
library(tidyr)
library(dplyr)

library(ggplot2)
library(fs)
library(readr)

# devtools::install_github("pachterlab/concordexR")
library(concordexR)

data_dir <- fs::path_wd("data")

# Load Data

Checkerboard data were generated using the script [here]() and stripe data were generated using the script [here](). The column `spatial_index_0.8` sorts the simulated cells by their spatial index, which differs from the order that they appear in the count matrix generated by [`Splatter`]().

In [ ]:
df_checker <- read_csv(path(data_dir,"checkerboard/cell_metadata.csv"))
df_stripe  <- read_csv(path(data_dir,"stripe/cell_metadata.csv"))

df_checker <- arrange(df_checker, spatial_index)
df_stripe <- arrange(df_stripe, spatial_index)

# Assignments for computing NbC matrix
labs_checker <- df_checker$Group
labs_stripe <- df_stripe$Group

# concordex Results

Now, compute concordex neighborhood consolidation matrix with cell type labels (checkerboard). Identify SHRs by providing a bluster clustering parameters object to the `BLUSPARAM` argument.

In [ ]:
# defaults to 30 nn
set.seed(5296)

coords <- df_checker[,c("row","col")]

cdx_checker <- calculateConcordex(
    x=coords,
    labels=labs_checker,
    BLUSPARAM=bluster::MbkmeansParam(2,50))

cdx_pred <- attr(cdx_checker, "shrs")
pl_data <- dplyr::mutate(coords, shr=cdx_pred)

In [ ]:
p <- ggplot(pl_data, aes(row, col, color=shr)) +
    geom_point(size=0.5) +
    theme_minimal() +
    scale_color_manual(values=c("#2CFFC6","#007756"))

p

In [ ]:
# defaults to 30 nn
set.seed(5296)

coords <- df_stripe[,c("row","col")]

cdx_stripe <- calculateConcordex(
    x=coords,
    labels=labs_stripe,
    BLUSPARAM=bluster::MbkmeansParam(4,50))

cdx_pred <- attr(cdx_stripe, "shrs")
pl_data <- dplyr::mutate(coords, shr=cdx_pred)

In [ ]:
p <- ggplot(pl_data, aes(row, col, color=shr)) +
    geom_point(size=0.5) +
    theme_minimal() +
    scale_color_manual(values=c("#2CFFC6","#007756","#D5C711","#005685"))

p